<h1>DeviceEngine Class

Dedicated engine for device data, inherited from Core Engine. Each DeviceEngine class object will represent a unique device with its own set of processing parameters and results.

In [1]:
from src.StreamPort.device.DeviceEngine import DeviceEngine
from src.StreamPort.core.ProcessingSettings import ProcessingSettings
from src.StreamPort.core.ProjectHeaders import ProjectHeaders

Create a new ProcessingSettings object 

In [2]:
settings = ProcessingSettings()

In [3]:
#specify path to get analyses from
base_dir = r'C:\Users\PC0118\Desktop\ExtractedSignals'

Creates an empty DeviceEngine object and prints it

In [4]:

dev = DeviceEngine(source = base_dir)
dev.print()


DeviceEngine 
  name: None 
  author: None 
  path: None 
  date: 2024-05-27 04:05:41.841303 
  analyses: 0 
  settings: 0 



Add project headers. They can be passed as ProjectHeaders objects or dict

In [5]:
dev.add_headers(headers = {'name': 'Pressure Curve Analysis', 'author': 'Sandeep H.'})
dev.print()


DeviceEngine 
  name: Pressure Curve Analysis 
  author: Sandeep H. 
  path: None 
  date: 2024-05-27 04:05:41.841303 
  analyses: 0 
  settings: 0 



Add processing settings

In [6]:
dev.add_settings(settings)
dev.print()


DeviceEngine 
  name: Pressure Curve Analysis 
  author: Sandeep H. 
  path: None 
  date: 2024-05-27 04:05:41.841303 
  analyses: 0 
  settings: 1 



<h1>DeviceAnalysis Class

Each DeviceAnalysis object is a child of the Analysis Class. It holds the details of an Analysis for each individual device.

In [7]:
from src.StreamPort.device.DeviceAnalysis import DeviceAnalysis

#Creates an empty DeviceAnalysis object and prints it
devAnalysis = DeviceAnalysis()
devAnalysis.print()


Analysis
  name: None
  replicate: None
  blank: None
  data:
  Empty



 
DeviceEngine's find_analyses() method returns a DeviceAnalysis Object or a list of DeviceAnalysis objects, besides printing the dataframes for each unique Method, paired with the metadata(Date, Runtime) for each curve.

This method makes use of the source variable to accept a path to a directory containing analyses as an argument and find analyses from the target path.

The path can refer to a directory containing data for a specific group of experiments "210812_Gem 2021-08-12 09-49-10" or one such experiment containing its own set of method-related analysis data "210812_Gem--005.D", "210812_Gem--007.D", ..



Read analysis objects from engine.

In [8]:
analyses = dev.find_analyses()

Sample - 02-blank : 
start date : 15:11:06 01/31/24
end date : 15:16:50 01/31/24
runtime : 0:05:44

Sample - 240131_Irino_Kali--003 : 
start date : 15:17:00 01/31/24
end date : 15:22:43 01/31/24
runtime : 0:05:43

Sample - 240131_Irino_Kali--004 : 
start date : 15:22:54 01/31/24
end date : 15:28:38 01/31/24
runtime : 0:05:44

Sample - 240131_Irino_Kali--005 : 
start date : 15:28:48 01/31/24
end date : 15:34:31 01/31/24
runtime : 0:05:43

Sample - 240131_Irino_Kali--006 : 
start date : 15:34:41 01/31/24
end date : 15:40:25 01/31/24
runtime : 0:05:44

Sample - 240131_Irino_Kali--007 : 
start date : 15:40:35 01/31/24
end date : 15:46:19 01/31/24
runtime : 0:05:44

Sample - 240131_Irino_Kali--008 : 
start date : 15:46:29 01/31/24
end date : 15:52:14 01/31/24
runtime : 0:05:45

Sample - 240131_Irino_Kali--009 : 
start date : 15:52:24 01/31/24
end date : 15:58:08 01/31/24
runtime : 0:05:44

Sample - 240131_Irino_Kali--010 : 
start date : 15:58:18 01/31/24
end date : 16:04:00 01/31/24
runtime

In [9]:
print(dev._method_ids)

['240131_Irino_Kali 2024-01-31 15-04-57', '210910_Doc 2021-09-10 17-13-54', '210827_Irino2 2021-08-27 10-26-23', '210819_Pac 2021-08-19 15-50-49', '210813_Pac 2021-08-13 10-37-27', '210813_5FU_210813 2021-08-13 13-33-00', '210813_5FU 2021-08-13 11-44-28', '210812_Pac 2021-08-12 10-30-07', '210812_Gem 2021-08-12 09-49-10']


In [10]:
print(dev._history)

{'240131_Irino_Kali 2024-01-31 15-04-57':      Time - 240131_Irino_Kali 2024-01-31 15-04-57  Sample - 02-blank  \
0                                           0.000             234.32   
1                                           0.005             237.00   
2                                           0.010             238.65   
3                                           0.015             240.47   
4                                           0.020             241.68   
..                                            ...                ...   
995                                         4.975             204.02   
996                                         4.980             204.15   
997                                         4.985             204.75   
998                                         4.990             204.88   
999                                         4.995             205.43   

     Sample - 240131_Irino_Kali--003  Sample - 240131_Irino_Kali--004  \
0                   

In [11]:
dev.print()


DeviceEngine 
  name: Pressure Curve Analysis 
  author: Sandeep H. 
  path: None 
  date: 2024-05-27 04:05:41.841303 
  analyses: 0 
  settings: 1 

240131_Irino_Kali 2024-01-31 15-04-57
     Time - 240131_Irino_Kali 2024-01-31 15-04-57  Sample - 02-blank  \
0                                           0.000             234.32   
1                                           0.005             237.00   
2                                           0.010             238.65   
3                                           0.015             240.47   
4                                           0.020             241.68   
..                                            ...                ...   
995                                         4.975             204.02   
996                                         4.980             204.15   
997                                         4.985             204.75   
998                                         4.990             204.88   
999                

In [12]:
dev.add_analyses(analyses)

In [13]:
for ana in dev._analyses:
    print("\n")
    ana.validate()
    print("Analysis Object : \n")
    print(f"Analysis : {ana.print()}")
    print("Data of Analysis : \n")
    print(ana.data)
    print("\n")



Analysis Object : 


Analysis
  name: Analysis - 240131_Irino_Kali 2024-01-31 15-04-57
  replicate: None
  blank: None
  data:
    Device Pressure Analysis - 15:11:06 01/31/24 (size 5)
    Device Pressure Analysis - 15:17:00 01/31/24 (size 5)
    Device Pressure Analysis - 15:22:54 01/31/24 (size 5)
    Device Pressure Analysis - 15:28:48 01/31/24 (size 5)
    Device Pressure Analysis - 15:34:41 01/31/24 (size 5)
    Device Pressure Analysis - 15:40:35 01/31/24 (size 5)
    Device Pressure Analysis - 15:46:29 01/31/24 (size 5)
    Device Pressure Analysis - 15:52:24 01/31/24 (size 5)
    Device Pressure Analysis - 15:58:18 01/31/24 (size 5)
    Device Pressure Analysis - 16:04:10 01/31/24 (size 5)
    Device Pressure Analysis - 15:05:08 01/31/24 (size 5)

Analysis : None
Data of Analysis : 

{'Device Pressure Analysis - 15:11:06 01/31/24': array(['Method : 240131_Irino_Kali 2024-01-31 15-04-57',
       'Sample : Sample - 02-blank', 'Start date : 15:11:06 01/31/24',
       'Runtime : 

Add analyses in the form of individual DeviceAnalysis objects or a list of such objects.

In [14]:
dev.print()


DeviceEngine 
  name: Pressure Curve Analysis 
  author: Sandeep H. 
  path: None 
  date: 2024-05-27 04:05:41.841303 
  analyses: 9 
  settings: 1 

240131_Irino_Kali 2024-01-31 15-04-57
     Time - 240131_Irino_Kali 2024-01-31 15-04-57  Sample - 02-blank  \
0                                           0.000             234.32   
1                                           0.005             237.00   
2                                           0.010             238.65   
3                                           0.015             240.47   
4                                           0.020             241.68   
..                                            ...                ...   
995                                         4.975             204.02   
996                                         4.980             204.15   
997                                         4.985             204.75   
998                                         4.990             204.88   
999                

Remove analysis by name of DeviceAnalysis object, index in list of objects, or list of names or indexes

In [15]:
dev.remove_analyses(0)
dev.print()


DeviceEngine 
  name: Pressure Curve Analysis 
  author: Sandeep H. 
  path: None 
  date: 2024-05-27 04:05:41.841303 
  analyses: 8 
  settings: 1 

240131_Irino_Kali 2024-01-31 15-04-57
     Time - 240131_Irino_Kali 2024-01-31 15-04-57  Sample - 02-blank  \
0                                           0.000             234.32   
1                                           0.005             237.00   
2                                           0.010             238.65   
3                                           0.015             240.47   
4                                           0.020             241.68   
..                                            ...                ...   
995                                         4.975             204.02   
996                                         4.980             204.15   
997                                         4.985             204.75   
998                                         4.990             204.88   
999                

Retrieve DeviceAnalysis Objects or list of such objects by object name, index

In [16]:
new_analyses = dev.get_analysis("Pac")

In [17]:
for ana in new_analyses:
    ana.print()


Analysis
  name: Analysis - 210819_Pac 2021-08-19 15-50-49
  replicate: None
  blank: None
  data:
    Device Pressure Analysis - 15:11:06 01/31/24 (size 5)
    Device Pressure Analysis - 15:17:00 01/31/24 (size 5)
    Device Pressure Analysis - 15:22:54 01/31/24 (size 5)
    Device Pressure Analysis - 15:28:48 01/31/24 (size 5)
    Device Pressure Analysis - 15:34:41 01/31/24 (size 5)
    Device Pressure Analysis - 15:40:35 01/31/24 (size 5)
    Device Pressure Analysis - 15:46:29 01/31/24 (size 5)
    Device Pressure Analysis - 15:52:24 01/31/24 (size 5)
    Device Pressure Analysis - 15:58:18 01/31/24 (size 5)
    Device Pressure Analysis - 16:04:10 01/31/24 (size 5)
    Device Pressure Analysis - 15:05:08 01/31/24 (size 5)
    Device Pressure Analysis - 11:13:50 09/13/21 (size 5)
    Device Pressure Analysis - 11:13:53 09/13/21 (size 5)
    Device Pressure Analysis - 11:13:56 09/13/21 (size 5)
    Device Pressure Analysis - 11:14:00 09/13/21 (size 5)
    Device Pressure Analysis -

In [18]:
print(dev._analyses)

[<src.StreamPort.device.DeviceAnalysis.DeviceAnalysis object at 0x000001FE0A942610>, <src.StreamPort.device.DeviceAnalysis.DeviceAnalysis object at 0x000001FE0A9423D0>, <src.StreamPort.device.DeviceAnalysis.DeviceAnalysis object at 0x000001FE0A95CBD0>, <src.StreamPort.device.DeviceAnalysis.DeviceAnalysis object at 0x000001FE0A952F10>, <src.StreamPort.device.DeviceAnalysis.DeviceAnalysis object at 0x000001FE023B7DD0>, <src.StreamPort.device.DeviceAnalysis.DeviceAnalysis object at 0x000001FE0A95ED90>, <src.StreamPort.device.DeviceAnalysis.DeviceAnalysis object at 0x000001FE08CAB650>, <src.StreamPort.device.DeviceAnalysis.DeviceAnalysis object at 0x000001FE09241050>]


In [19]:
for i in dev._analyses:
    i.print()


Analysis
  name: Analysis - 210910_Doc 2021-09-10 17-13-54
  replicate: None
  blank: None
  data:
    Device Pressure Analysis - 15:11:06 01/31/24 (size 5)
    Device Pressure Analysis - 15:17:00 01/31/24 (size 5)
    Device Pressure Analysis - 15:22:54 01/31/24 (size 5)
    Device Pressure Analysis - 15:28:48 01/31/24 (size 5)
    Device Pressure Analysis - 15:34:41 01/31/24 (size 5)
    Device Pressure Analysis - 15:40:35 01/31/24 (size 5)
    Device Pressure Analysis - 15:46:29 01/31/24 (size 5)
    Device Pressure Analysis - 15:52:24 01/31/24 (size 5)
    Device Pressure Analysis - 15:58:18 01/31/24 (size 5)
    Device Pressure Analysis - 16:04:10 01/31/24 (size 5)
    Device Pressure Analysis - 15:05:08 01/31/24 (size 5)
    Device Pressure Analysis - 11:13:50 09/13/21 (size 5)
    Device Pressure Analysis - 11:13:53 09/13/21 (size 5)
    Device Pressure Analysis - 11:13:56 09/13/21 (size 5)
    Device Pressure Analysis - 11:14:00 09/13/21 (size 5)
    Device Pressure Analysis -

In [20]:
dev.plot_analyses(2)

In [23]:
dev.plot_analyses([3, 4, 5])

In [21]:
dev.remove_analyses(new_analyses)

In [22]:
dev.print()


DeviceEngine 
  name: Pressure Curve Analysis 
  author: Sandeep H. 
  path: None 
  date: 2024-05-27 04:05:41.841303 
  analyses: 8 
  settings: 1 

240131_Irino_Kali 2024-01-31 15-04-57
     Time - 240131_Irino_Kali 2024-01-31 15-04-57  Sample - 02-blank  \
0                                           0.000             234.32   
1                                           0.005             237.00   
2                                           0.010             238.65   
3                                           0.015             240.47   
4                                           0.020             241.68   
..                                            ...                ...   
995                                         4.975             204.02   
996                                         4.980             204.15   
997                                         4.985             204.75   
998                                         4.990             204.88   
999                